## Find bad MEG channels
For each participant session, manually inspect the data and identify bad MEG channels to exclude them before the rest of preprocessing. **Every session should have the same bad channels across all files.** The list of bad channels is saved to the text file `{subject}_bads.txt`.

It's recommended to run MNE-Python locally on a Mac/Windows so you don't need to use graphics forwarding from Linux to non-Linux. **If you ignore this advice and run this notebook remotely, you'll need to enable a display environment, since Qt requires a GUI backend**
- Option 1: Use X11 Forwarding via SSH -- run [XQuartz](https://www.xquartz.org/) (Mac) or [X11](https://pkgs.org/download/x11-common) (Linux) on your local machine
- Option 2: Use a Virtual Display (like [Xvfb](https://pkgs.org/download/Xvfb))

In [1]:
# load packages
import os
import time
import mne
from mne.io import read_raw_fif
#%matplotlib qt

In [2]:
# session info
subject = 'nbl_005'
session = '01'
file = 'bigfrank_1'

# filter params
filt_l = 1
filt_notch = 60
filt_h = 200

In [3]:
# file paths
processed_meg_dir = '/mnt/sphere/nbl/processed_meg/'
sub_ses = os.path.join(subject, 'ses-'+session)
recording_fname = f"{subject}_{file}_raw.fif"
raw_fname = os.path.join(processed_meg_dir, sub_ses, recording_fname)
bads_list_fname = os.path.join(processed_meg_dir, sub_ses, subject+'_bads.txt')

In [4]:
print (f"Reading raw file for {subject}...")
t = time.time()
raw = read_raw_fif(raw_fname, preload=True, verbose=True)
elapsed_readraw = time.time() - t
print(f"Loaded in {elapsed_readraw:.1f} seconds.\n")
print ("Filtering data...")
raw = raw.filter(filt_l, filt_h, n_jobs=-1)

Reading raw file for nbl_005...
Opening raw data file /mnt/sphere/nbl/processed_meg/nbl_005/ses-01/nbl_005_bigfrank_1_raw.fif...
    Read a total of 13 projection items:
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
    Range : 29000 ... 1580999 =     29.000 ...  1580.999 secs
Ready.
Opening raw data file /mnt/sphere/nbl/processed_meg/nbl_005/ses-01/nbl_005_bigfrank_1_raw-1.fif...
    Read a total of 13 pr

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 306 out of 306 | elapsed:   23.5s finished


In [11]:
# pick bad channels and annotate
# %matplotlib inline
raw.plot()

/mnt/cube/blang/envs/mne/lib/python3.12/site-packages/pyqtgraph/GraphicsScene/GraphicsScene.py:378: RuntimeWarning: Projection vector 'generated with dossp-2.1' has been reduced to 66.18% of its original magnitude by subselecting 304/204 of the original channels. If the ignored channels were bad during SSP computation, we recommend recomputing proj (via compute_proj_raw or related functions) with the bad channels properly marked, because computing SSP with bad channels present in the data but unmarked is dangerous (it can bias the PCA used by SSP). On the other hand, if you know that all channels were good during SSP computation, you can safely use info.normalize_proj() to suppress this warning during projection.
  item.mouseClickEvent(ev)
/mnt/cube/blang/envs/mne/lib/python3.12/site-packages/pyqtgraph/GraphicsScene/GraphicsScene.py:378: RuntimeWarning: Projection vector 'generated with dossp-2.1' has been reduced to 66.18% of its original magnitude by subselecting 304/204 of the origi

Channels marked as bad:
['MEG1132', 'MEG1543', 'MEG1843']


In [12]:
# save a list of the bad channels
with open(bads_list_fname, 'w') as f:
    for line in raw.info['bads']:
        f.write(f"{line}\n")